In [ ]:
#code for creating the target output
import pandas as pd

# Assuming your dataset is stored in a DataFrame called df
# Make sure your 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Sort the DataFrame by 'permno' and 'date' in ascending order
df.sort_values(by=['permno', 'date'], inplace=True)

# Define a function to find the closest available price within a range of days
def find_closest_price(row, prices):
    target_date = row['date']
    permno = row['permno']
    price_index = prices.index
    closest_date = price_index[price_index <= target_date][-1]
    return prices.loc[closest_date, permno] if closest_date in prices.index else None

# Group the DataFrame by 'permno' and create a dictionary of DataFrames for each group
price_groups = {permno: group.set_index('date')['price'] for permno, group in df.groupby('permno')}

# Apply the function to find the closest price for each row
df['price_90_days_ago'] = df.apply(find_closest_price, args=(price_groups,), axis=1)

# Calculate the price difference between today and 90 days before
df['price_diff'] = df['price'] - df['price_90_days_ago']

# Define a function to assign 1 if price difference is positive, -1 otherwise
def assign_target(x):
    if x > 0:
        return 1
    else:
        return -1

# Apply the function to create the 'target' column
df['target'] = df['price_diff'].apply(assign_target)

# Drop intermediate columns if you don't need them anymore
df.drop(columns=['price_90_days_ago', 'price_diff'], inplace=True)

# Reset index if needed
df.reset_index(drop=True, inplace=True)

# Print or save the updated DataFrame
print(df)
